In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import geopandas as gpd
import osmnx as ox
import folium
from src.data_processing.bikepaths import get_graph_without_bikepaths
from src.data_processing.accident_edge_metrics import compute_danger_metric
from src import STORAGE_PATH
import matplotlib
import momepy

target_crs = 'EPSG:2180'
map_crs = 'EPSG:4326'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load graphs

In [ ]:
DATA_DIRECTORY = STORAGE_PATH / 'data'

accidents = gpd.read_file(DATA_DIRECTORY / 'accidents.geojson').to_crs(target_crs)
bike_accidents = accidents.query('POJ_ROWER > 0')
trasy_rowerowe  = gpd.read_file(DATA_DIRECTORY / 'bikeroads.geojson').to_crs(target_crs)
# trasy_rowerowe = trasy_rowerowe.query('TYP != "strefa ruchu uspokojonego 20 i 30 km/h"')
trasy_rowerowe_graph = momepy.gdf_to_nx(trasy_rowerowe.to_crs(map_crs), approach='primal')

In [4]:
distance = 6000
simplify = True
center = (51.107883, 17.038538)

roads = ox.graph_from_point(
    center,
    dist=distance,
    simplify=simplify,
    network_type='drive',
)
roads = ox.project_graph(roads, target_crs)
roads = ox.consolidate_intersections(roads, tolerance=15, dead_ends=True).to_undirected()

/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:499: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subcluster_centroid = node_points.loc[wcc].unary_union.centroid
/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:500: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "x"] = subcluster_centroid.x
/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:501: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "y"] = subcluster_centroid.y
/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:503: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead

In [6]:
roads_without_bikeroads = get_graph_without_bikepaths(roads, trasy_rowerowe, min_road_to_bike_road_dist=20)
danger_metric = compute_danger_metric(roads_without_bikeroads, roads, bike_accidents)

In [10]:
def rgb2hex(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))


roads_proj = ox.project_graph(roads, map_crs)
node_to_location = {k: (data['y'], data['x']) for k, data in roads_proj.nodes(data=True)}

folium_map = folium.Map(
    location=(51.107883, 17.038538),
    zoom_start=12,
    tiles='CartoDBPositron',
)

lines_layer = folium.FeatureGroup('All roads')
for start, end, data in roads_proj.edges(data=True):
    start_position = node_to_location[start]
    end_position = node_to_location[end]

    folium.PolyLine(
        [start_position, end_position],
        weight=1,
        color='#03a9f4',
    ).add_to(lines_layer)
lines_layer.add_to(folium_map)


cmap = matplotlib.cm.get_cmap('Reds')
lines_layer = folium.FeatureGroup('No bike lines')
for start, end, idx in ox.project_graph(roads_without_bikeroads, map_crs).edges:
    start_position = node_to_location[start]
    end_position = node_to_location[end]

    metric = danger_metric[(start, end, idx)]

    color = rgb2hex(*cmap(metric)[:-1])
    folium.PolyLine(
        [start_position, end_position],
        weight=3,
        color=color,
    ).add_to(lines_layer)
lines_layer.add_to(folium_map)

lines_layer = folium.FeatureGroup('Bike lines')
for start, end, data in trasy_rowerowe_graph.edges(data=True):
    weight = 1
    color = '#aed581'
    x1, y1 = start
    x2, y2 = end
    folium.PolyLine(
        [(y1, x1), (y2, x2)],
        weight=weight,
        color=color,
    ).add_to(lines_layer)

lines_layer.add_to(folium_map)
folium.LayerControl().add_to(folium_map)

folium_map.save('map.html')